# Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Import data bases

In [12]:
df_pass = pd.read_csv('./data/base_modelis_pass_det.csv',sep = ';')

In [18]:
df_pass.shape

(329064, 90)

# To do list

Analyse exploratoire : distrib var quali/quanti, visualisation NaN, corrélations, doublons, outliers

Analyse + profonde : que prédire, quelles vars sont pertinentes, diff entre les 2 df, liste questions à poser

Axel : distrib quanti, correlations, outliers

Baheux : distrib quali ,NaN , doublons

In [24]:
round(df_pass.isna().sum()/len(df_pass)*100,2)

cd_op                  0.00
date_entree_defaut     0.00
date_sortie_defaut    95.87
arrete                 0.00
fl_pass_DET            0.00
                      ...  
PTG                    8.84
AGE_EMP1               0.00
AGE_EMP2              40.82
TX_ENDET               6.06
MT_REST_VIVRE          7.71
Length: 90, dtype: float64